In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Add, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

ratings = pd.read_csv('ml-100k/ratings.csv', sep=',',encoding='latin-1')

In [2]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [29]:
u_unique = ratings.userId.unique()
user2Id = { userId : index for index, userId in enumerate(u_unique)}
ratings.userId = ratings.userId.apply(lambda x: user2Id[x])

m_unique = ratings.movieId.unique()
movie2Id = { movieId : index for index, movieId in enumerate(m_unique)}
ratings.movieId = ratings.movieId.apply(lambda x: movie2Id[x])

In [31]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(610, 9724)

In [5]:
n_factors = 50

In [6]:
np.random.seed = 42

In [7]:
msk = np.random.rand(len(ratings)) < 0.8

In [32]:
trn = ratings[msk]
val = ratings[~msk]

# Dot Product Matrix Factorization

In [34]:
user_in = Input(shape=(1,), dtype='int64', name='user_input')
u = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-5))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_input')
m = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-5))(movie_in)

In [35]:
x = Dot(axes=-1)([u,m])
x = Flatten()(x)
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [36]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=6,
          validation_data=([val.userId, val.userId], val.rating))

Train on 81021 samples, validate on 19815 samples
Epoch 1/6
81021/81021 [==============================] - 19s 232us/sample - loss: 9.7629 - val_loss: 4.6226
Epoch 2/6
81021/81021 [==============================] - 20s 244us/sample - loss: 2.2742 - val_loss: 2.1929
Epoch 3/6
81021/81021 [==============================] - 21s 264us/sample - loss: 1.3262 - val_loss: 1.7349
Epoch 4/6
81021/81021 [==============================] - 18s 218us/sample - loss: 1.0496 - val_loss: 1.5227
Epoch 5/6
81021/81021 [==============================] - 18s 225us/sample - loss: 0.9213 - val_loss: 1.4646
Epoch 6/6
81021/81021 [==============================] - 19s 231us/sample - loss: 0.8427 - val_loss: 1.4284


# Bias Model

In [12]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

In [37]:
user_in, u = embedding_input('user_input', n_users, n_factors, 1e-5)
movie_in, m = embedding_input('movie_input', n_movies, n_factors, 1e-5)

In [38]:
def create_bias(inp, n_in):
    return Flatten()(Embedding(n_in, 1, input_length=1)(inp))

In [39]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [40]:
x = Dot(axes=-1)([u,m])
x = Flatten()(x)
x = Add()([x, ub])
x = Add()([x, mb])
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [41]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=6,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 81021 samples, validate on 19815 samples
Epoch 1/6
81021/81021 [==============================] - 20s 246us/sample - loss: 8.5263 - val_loss: 2.9365
Epoch 2/6
81021/81021 [==============================] - 19s 235us/sample - loss: 1.8265 - val_loss: 1.5304
Epoch 3/6
81021/81021 [==============================] - 22s 265us/sample - loss: 1.1189 - val_loss: 1.2717
Epoch 4/6
81021/81021 [==============================] - 19s 234us/sample - loss: 0.9181 - val_loss: 1.1904
Epoch 5/6
81021/81021 [==============================] - 19s 228us/sample - loss: 0.8202 - val_loss: 1.1566
Epoch 6/6
81021/81021 [==============================] - 20s 252us/sample - loss: 0.7520 - val_loss: 1.1414


# Neural Net

In [45]:
user_in, u = embedding_input('user_input', n_users , n_factors, 1e-5)
movie_in, m = embedding_input('movie_input', n_movies, n_factors, 1e-5)

In [46]:
x = Concatenate()([u, m])
x = Flatten()(x)
x = Dense(units=100, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(1)(x)
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [47]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=5,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 81021 samples, validate on 19815 samples
Epoch 1/5
81021/81021 [==============================] - 20s 244us/sample - loss: 1.6328 - val_loss: 0.7927
Epoch 2/5
81021/81021 [==============================] - 19s 235us/sample - loss: 0.9404 - val_loss: 0.7761
Epoch 3/5
81021/81021 [==============================] - 19s 237us/sample - loss: 0.8743 - val_loss: 0.7750
Epoch 4/5
81021/81021 [==============================] - 21s 264us/sample - loss: 0.8209 - val_loss: 0.7692
Epoch 5/5
81021/81021 [==============================] - 22s 270us/sample - loss: 0.7784 - val_loss: 0.7726
